## Import libraries

In [49]:
import io
import itertools

import numpy as np
import sklearn.metrics

import tensorflow as tf
from tensorboard.plugins.hparams import api as hp

import matplotlib.pyplot as plt

## Load and preprocess data

In [50]:
train_data = np.load('dataset/Trousers & Jeans - All - Train.npz')
validation_data = np.load('dataset/Trousers & Jeans - All - Validation.npz')
test_data = np.load('dataset/Trousers & Jeans - All - Test.npz')

In [51]:
list(train_data.keys())

['images', 'labels', 'genders']

In [52]:
images_train, labels_train = train_data['images'], train_data['labels']
images_val, labels_val = validation_data['images'], validation_data['labels']
images_test, labels_test = test_data['images'], test_data['labels']

In [53]:
np.unique(labels_train)

array([0, 1, 2, 3])

In [54]:
images_train.shape

(4004, 120, 90, 3)

In [55]:
labels_train.shape

(4004,)

In [56]:
images_val.shape

(500, 120, 90, 3)

In [57]:
labels_val.shape

(500,)

In [58]:
images_test.shape

(500, 120, 90, 3)

In [59]:
labels_test.shape

(500,)

In [60]:
# standardize 
images_train = images_train/255.0
images_val = images_val/255.0
images_test = images_test/255.0

## Create the model!

### Define hyparameters

In [61]:
EPOCHS = 15
BATCH_SIZE = 64

In [62]:
HP_DROPOUT_RATE = hp.HParam('dropout_rate', hp.Discrete([0.1,0.2,0.3,0.4,0.5,0.6,0.7]))

In [63]:
METRIC = 'accuracy'

with tf.summary.create_file_writer(r'Logs/Model 5 (Trousers&Jeans_All_Dropout)/hparam_tuning/').as_default():
    hp.hparams_config(
        hparams = [HP_DROPOUT_RATE],
        metrics = [hp.Metric(METRIC, display_name='accuracy')]
    )

### Model architecture

In [64]:
def train_test_model(hparams, session_num):
    model = tf.keras.Sequential([
        # we choose the valueus for kernel_num, kernel_size, and dense from the best of the hyperparameter tuning we do in model 3
        tf.keras.layers.Conv2D(96, 7, activation = 'relu', input_shape = (120,90,3)),
        tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
        tf.keras.layers.Conv2D(96, 3, activation = 'relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(512, activation = 'relu'),
        tf.keras.layers.Dropout(hparams[HP_DROPOUT_RATE]),
        tf.keras.layers.Dense(4, activation='softmax')
    ])
    
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
    model.compile(optimizer = 'adam', loss = loss_fn, metrics = ['accuracy'])
    
    log_dir = "Logs\\Model 5 (Trousers&Jeans_All_Dropout)\\fit\\" + f"run-{session_num}"
    
    # functions for creating confusion matrix
    def plot_confusion_matrix(cm, class_names):
        """
        Returns a matplotlib figure containing the plotted confusion matrix.

        Args:
        cm (array, shape = [n, n]): a confusion matrix of integer classes
        class_names (array, shape = [n]): String names of the integer classes
        """
        figure = plt.figure(figsize=(12, 12))
        plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
        plt.title("Confusion matrix")
        plt.colorbar()
        tick_marks = np.arange(len(class_names))
        plt.xticks(tick_marks, class_names, rotation=45)
        plt.yticks(tick_marks, class_names)

        # Normalize the confusion matrix.
        cm = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)

        # Use white text if squares are dark; otherwise black.
        threshold = cm.max() / 2.
        for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
            color = "white" if cm[i, j] > threshold else "black"
            plt.text(j, i, cm[i, j], horizontalalignment="center", color=color)

        plt.tight_layout()
        plt.ylabel('True label')
        plt.xlabel('Predicted label')

        return figure

    def plot_to_image(figure):
        """Converts the matplotlib plot specified by 'figure' to a PNG image and
        returns it. The supplied figure is closed and inaccessible after this call."""

        # Save the plot to a PNG in memory.
        buf = io.BytesIO()
        plt.savefig(buf, format='png')

        # Closing the figure prevents it from being displayed directly inside the notebook.
        plt.close(figure)

        buf.seek(0)

        # Convert PNG buffer to TF image
        image = tf.image.decode_png(buf.getvalue(), channels=4)

        # Add the batch dimension
        image = tf.expand_dims(image, 0)

        return image

    # Define a file writer variable for logging purposes
    file_writer_cm = tf.summary.create_file_writer(log_dir + '/cm')

    def log_confusion_matrix(epoch, logs):
        # Use the model to predict the values from the validation dataset.
        test_pred_raw = model.predict(images_val)
        test_pred = np.argmax(test_pred_raw, axis=1)

        # Calculate the confusion matrix.
        cm = sklearn.metrics.confusion_matrix(labels_val, test_pred)

        # Log the confusion matrix as an image summary.
        figure = plot_confusion_matrix(cm, class_names=['Trousers Male','Jeans Male','Trousers Female','Jeans Female'])
        cm_image = plot_to_image(figure)

        # Log the confusion matrix as an image summary.
        with file_writer_cm.as_default():
            tf.summary.image("Confusion Matrix", cm_image, step=epoch)
            
    # callbacks
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 2, restore_best_weights = True)
    cm_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end = log_confusion_matrix)
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq = 1, profile_batch = 0)
    
    model.fit(images_train,
              labels_train,
              epochs = EPOCHS,
              batch_size = BATCH_SIZE,
              callbacks = [tensorboard_callback, cm_callback, early_stopping],
              validation_data = (images_val, labels_val),
              verbose = 1)
    
    _, accuracy = model.evaluate(images_val, labels_val)
    
    model.save(r'saved_models\Model 5 (Trousers&Jeans_All_Dropout)\Run-{}'.format(session_num))
    
    return accuracy

In [65]:
def run(log_dir, hparams, session_num):
    with tf.summary.create_file_writer(log_dir).as_default():
        hp.hparams(hparams)  # record the values used in this trial
        accuracy = train_test_model(hparams, session_num)
        tf.summary.scalar(METRIC,accuracy,step=1)

In [66]:
session_num = 1
for dropout_rate in HP_DROPOUT_RATE.domain.values:
    hparams = {
        HP_DROPOUT_RATE : dropout_rate
    }
    run_name = f'run-{session_num}'
    print('--- Starting Trial:',run_name)
    print({h.name: hparams[h] for h in hparams})
    run("Logs/Model 5 (Trousers&Jeans_All_Dropout)/hparam_tuning/" + run_name, hparams, session_num)

    session_num += 1

--- Starting Trial: run-1
{'dropout_rate': 0.1}
Epoch 1/15
63/63 [==============================] - 72s 1s/step - loss: 1.4386 - accuracy: 0.4675 - val_loss: 1.0663 - val_accuracy: 0.5480
Epoch 2/15
63/63 [==============================] - 71s 1s/step - loss: 0.9647 - accuracy: 0.6109 - val_loss: 1.1192 - val_accuracy: 0.5540
Epoch 3/15
63/63 [==============================] - 72s 1s/step - loss: 0.8326 - accuracy: 0.6618 - val_loss: 0.9273 - val_accuracy: 0.6380
Epoch 4/15
63/63 [==============================] - 79s 1s/step - loss: 0.7750 - accuracy: 0.6813 - val_loss: 0.8741 - val_accuracy: 0.6520
Epoch 5/15
63/63 [==============================] - 76s 1s/step - loss: 0.7033 - accuracy: 0.7113 - val_loss: 0.8260 - val_accuracy: 0.6520
Epoch 6/15
63/63 [==============================] - 75s 1s/step - loss: 0.6578 - accuracy: 0.7230 - val_loss: 0.9030 - val_accuracy: 0.6640
Epoch 7/15
16/16 [==============================] - 2s 139ms/step - loss: 0.8260 - accuracy: 0.6520
Instructions

63/63 [==============================] - 72s 1s/step - loss: 0.7039 - accuracy: 0.7088 - val_loss: 0.8020 - val_accuracy: 0.6900
Epoch 9/15
63/63 [==============================] - 71s 1s/step - loss: 0.6648 - accuracy: 0.7275 - val_loss: 0.8046 - val_accuracy: 0.6680
Epoch 10/15
63/63 [==============================] - 71s 1s/step - loss: 0.6604 - accuracy: 0.7203 - val_loss: 0.7845 - val_accuracy: 0.6980
Epoch 11/15
63/63 [==============================] - 71s 1s/step - loss: 0.6115 - accuracy: 0.7483 - val_loss: 0.7490 - val_accuracy: 0.7060
Epoch 12/15
63/63 [==============================] - 72s 1s/step - loss: 0.5968 - accuracy: 0.7557 - val_loss: 0.7361 - val_accuracy: 0.7100
Epoch 13/15
63/63 [==============================] - 71s 1s/step - loss: 0.5604 - accuracy: 0.7770 - val_loss: 0.7705 - val_accuracy: 0.7040
Epoch 14/15
16/16 [==============================] - 2s 133ms/step - loss: 0.7361 - accuracy: 0.7100
INFO:tensorflow:Assets written to: saved_models\Model 5 (Trousers&